In [1]:
pip install datasets pandas numpy matplotlib seaborn krippendorff


Note: you may need to restart the kernel to use updated packages.


In [2]:
import krippendorff
from datasets import load_dataset
import pandas as pd
import numpy as np

from pandas.core.computation.check import NUMEXPR_INSTALLED

/opt/anaconda3/lib/python3.13/site-packages/pandas/core/computation/expressions.py:22: UserWarning: Pandas requires version '2.10.2' or newer of 'numexpr' (version '2.10.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
dataset = load_dataset("ucberkeley-dlab/measuring-hate-speech")


In [4]:
dataset["train"][0]


{'comment_id': 47777,
 'annotator_id': 10873,
 'platform': 3,
 'sentiment': 0.0,
 'respect': 0.0,
 'insult': 0.0,
 'humiliate': 0.0,
 'status': 2.0,
 'dehumanize': 0.0,
 'violence': 0.0,
 'genocide': 0.0,
 'attack_defend': 0.0,
 'hatespeech': 0.0,
 'hate_speech_score': -3.9,
 'text': 'Yes indeed. She sort of reminds me of the elder lady that played the part in the movie "Titanic" who was telling her story!!! And I wouldn\'t have wanted to cover who I really am!! I would be proud!!!! WE should be proud of our race no matter what it is!!',
 'infitms': 0.81,
 'outfitms': 1.88,
 'annotator_severity': 0.36,
 'std_err': 0.34,
 'annotator_infitms': 1.35,
 'annotator_outfitms': 1.23,
 'hypothesis': -1.1301777576839678,
 'target_race_asian': True,
 'target_race_black': True,
 'target_race_latinx': True,
 'target_race_middle_eastern': True,
 'target_race_native_american': True,
 'target_race_pacific_islander': True,
 'target_race_white': True,
 'target_race_other': False,
 'target_race': True,
 

In [5]:
df = pd.DataFrame(dataset["train"])

 # Checking for duplicates

#### Checking exact duplicates

In [6]:
df.duplicated().sum()


np.int64(0)

#### Checking duplicates ignoring the index (same content, repeated)

In [7]:
df.reset_index(drop=True).duplicated().sum()


np.int64(0)

#### Checking duplicates on comment id and annotator id

In [8]:
df.duplicated(subset=["comment_id", "annotator_id"]).sum()


np.int64(0)

#### Checking duplicates on text and annotator id

In [9]:
df.duplicated(subset=["text", "annotator_id"]).sum()


np.int64(0)

## Data Splitting (70% Training, 20% Testing, 10%Validation)

In [10]:
from sklearn.model_selection import train_test_split

comment_ids = df["comment_id"].unique()

In [11]:
comment_ids

array([47777, 39773, 47101, ..., 30588, 21008, 37080])

In [12]:
train_ids, temp_ids = train_test_split(
    comment_ids,
    test_size=0.30,
    random_state=42
)

test_ids, val_ids = train_test_split(
    temp_ids,
    test_size=1/3,
    random_state=42
)


In [13]:
train_df = df[df["comment_id"].isin(train_ids)]
test_df  = df[df["comment_id"].isin(test_ids)]
val_df   = df[df["comment_id"].isin(val_ids)]


In [14]:
print("Original rows:", len(df))
print("Train rows:", len(train_df))
print("Test rows:", len(test_df))
print("Validation rows:", len(val_df))
print("Total after split:", len(train_df) + len(test_df) + len(val_df)) 

Original rows: 135556
Train rows: 93935
Test rows: 25413
Validation rows: 16208
Total after split: 135556


In [15]:
total_rows = len(df)

print("Train %:", round(len(train_df) / total_rows * 100, 2))
print("Test %:", round(len(test_df) / total_rows * 100, 2))
print("Validation %:", round(len(val_df) / total_rows * 100, 2))


Train %: 69.3
Test %: 18.75
Validation %: 11.96


*Note - Splitting is done based on the comments. After splitting, annotators are added for that comment and hence the variation in dataset splitting percentage (which is almost around 70-20-10).*

## Checking Data Leakage 

#### Checking comment-level leakage

In [16]:
train_ids = set(train_df["comment_id"])
test_ids  = set(test_df["comment_id"])
val_ids   = set(val_df["comment_id"])

print("Train and Test:", len(train_ids & test_ids))
print("Train and Val:", len(train_ids & val_ids))
print("Test and Val:", len(test_ids & val_ids))


Train and Test: 0
Train and Val: 0
Test and Val: 0


#### Checking annotation-level leakage

In [17]:
train_pairs = set(zip(train_df["comment_id"], train_df["annotator_id"]))
test_pairs  = set(zip(test_df["comment_id"], test_df["annotator_id"]))
val_pairs   = set(zip(val_df["comment_id"], val_df["annotator_id"]))

print("Train and Test (pairs):", len(train_pairs & test_pairs))
print("Train and Val (pairs):", len(train_pairs & val_pairs))
print("Test and Val (pairs):", len(test_pairs & val_pairs))


Train and Test (pairs): 0
Train and Val (pairs): 0
Test and Val (pairs): 0


#### Checking no rows were lost

In [18]:
print("Original rows:", len(df))
print("After split:", len(train_df) + len(test_df) + len(val_df))


Original rows: 135556
After split: 135556


# New Splitting Criteria

In [19]:

annotator_religions = [
    "annotator_religion_muslim",
    "annotator_religion_christian",
    "annotator_religion_jewish",
    "annotator_religion_atheist",
    "annotator_religion_buddhist"
]

target_religions = [
    "target_religion_muslim",
    "target_religion_christian",
    "target_religion_jewish",
    "target_religion_atheist",
    "target_religion_buddhist"
]

df["hate_speech_score"] = pd.to_numeric(df["hate_speech_score"], errors="coerce")
df["is_hate_speech"] = df["hate_speech_score"] > 0.5


In [20]:
def make_hate_nonhate_table(dataframe):
    table = pd.DataFrame()

    for a_col in annotator_religions:
        row_name = a_col.replace("annotator_religion_", "").capitalize() + " annotators"
        row_data = {}

        annotator_subset = dataframe[dataframe[a_col] == 1]

        for t_col in target_religions:
            target_name = t_col.replace("target_religion_", "").capitalize()
            subset = annotator_subset[annotator_subset[t_col] == 1]

            total = len(subset)

            if total > 0:
                hate_pct = round(subset["is_hate_speech"].mean() * 100, 2)
                non_hate_pct = round(100 - hate_pct, 2)
            else:
                hate_pct = np.nan
                non_hate_pct = np.nan

            row_data[f"% Hate → {target_name}"] = hate_pct
            row_data[f"% Non-Hate → {target_name}"] = non_hate_pct

        table = pd.concat([table, pd.DataFrame(row_data, index=[row_name])])

    return table


In [21]:
print("BEFORE SPLITTING (FULL DATASET)")
print("Total rows:", len(df))
print("Unique comments:", df["comment_id"].nunique())

overall_hate = round(df["is_hate_speech"].mean() * 100, 2)
print("Overall Hate %:", overall_hate)
print("Overall Non-Hate %:", round(100 - overall_hate, 2))

full_table = make_hate_nonhate_table(df)
full_table


BEFORE SPLITTING (FULL DATASET)
Total rows: 135556
Unique comments: 39565
Overall Hate %: 36.18
Overall Non-Hate %: 63.82


,% Hate → Muslim,% Non-Hate → Muslim,% Hate → Christian,% Non-Hate → Christian,% Hate → Jewish,% Non-Hate → Jewish,% Hate → Atheist,% Non-Hate → Atheist,% Hate → Buddhist,% Non-Hate → Buddhist
Muslim annotators,33.02,66.98,12.73,87.27,61.54,38.46,26.67,73.33,25.00,75.00
Christian annotators,40.79,59.21,13.29,86.71,63.01,36.99,18.93,81.07,23.80,76.20
Jewish annotators,45.45,54.55,13.13,86.87,69.13,30.87,35.71,64.29,33.33,66.67
Atheist annotators,43.23,56.77,10.27,89.73,66.23,33.77,18.54,81.46,20.00,80.00
Buddhist annotators,39.15,60.85,12.95,87.05,62.00,38.00,5.88,94.12,25.00,75.00


In [22]:
comment_level = (
    df.groupby("comment_id")
      .agg(mean_score=("hate_speech_score", "mean"))
      .reset_index()
)

comment_level["label"] = (comment_level["mean_score"] > 0.5).astype(int)

comment_ids = comment_level["comment_id"]
comment_labels = comment_level["label"]

train_ids, temp_ids = train_test_split(
    comment_ids,
    test_size=0.30,
    random_state=42,
    stratify=comment_labels
)

temp_df = comment_level[comment_level["comment_id"].isin(temp_ids)]

test_ids, val_ids = train_test_split(
    temp_df["comment_id"],
    test_size=1/3,               # val = 10%, test = 20%
    random_state=42,
    stratify=temp_df["label"]
)
train_df = df[df["comment_id"].isin(train_ids)]
test_df  = df[df["comment_id"].isin(test_ids)]
val_df   = df[df["comment_id"].isin(val_ids)]


In [23]:
print("\nAFTER SPLITTING ")
print("Original rows:", len(df))
print("Train rows:", len(train_df))
print("Test rows:", len(test_df))
print("Val rows:", len(val_df))
print("Total after split:", len(train_df) + len(test_df) + len(val_df))

print("\nRow %:")
print("Train %:", round(len(train_df)/len(df)*100, 2))
print("Test %:", round(len(test_df)/len(df)*100, 2))
print("Val %:", round(len(val_df)/len(df)*100, 2))

print("\nComment %:")
print("Train comment %:", round(train_df["comment_id"].nunique()/df["comment_id"].nunique()*100, 2))
print("Test comment %:", round(test_df["comment_id"].nunique()/df["comment_id"].nunique()*100, 2))
print("Val comment %:", round(val_df["comment_id"].nunique()/df["comment_id"].nunique()*100, 2))

print("\nOverall Hate/Non-Hate % by split:")
for name, d in [("Train", train_df), ("Test", test_df), ("Val", val_df)]:
    hate = round(d["is_hate_speech"].mean()*100, 2)
    print(name, "Hate %:", hate, "| Non-Hate %:", round(100-hate, 2))



AFTER SPLITTING 
Original rows: 135556
Train rows: 94851
Test rows: 26632
Val rows: 14073
Total after split: 135556

Row %:
Train %: 69.97
Test %: 19.65
Val %: 10.38

Comment %:
Train comment %: 70.0
Test comment %: 20.0
Val comment %: 10.0

Overall Hate/Non-Hate % by split:
Train Hate %: 37.13 | Non-Hate %: 62.87
Test Hate %: 32.59 | Non-Hate %: 67.41
Val Hate %: 36.59 | Non-Hate %: 63.41


In [24]:
print("\n TRAIN TABLE ")
train_table = make_hate_nonhate_table(train_df)
train_table



 TRAIN TABLE 


,% Hate → Muslim,% Non-Hate → Muslim,% Hate → Christian,% Non-Hate → Christian,% Hate → Jewish,% Non-Hate → Jewish,% Hate → Atheist,% Non-Hate → Atheist,% Hate → Buddhist,% Non-Hate → Buddhist
Muslim annotators,41.33,58.67,15.79,84.21,54.84,45.16,27.27,72.73,28.57,71.43
Christian annotators,48.27,51.73,17.45,82.55,60.78,39.22,21.59,78.41,27.16,72.84
Jewish annotators,56.35,43.65,13.21,86.79,65.22,34.78,40.00,60.00,25.00,75.00
Atheist annotators,50.31,49.69,12.05,87.95,62.40,37.60,17.80,82.20,16.67,83.33
Buddhist annotators,44.30,55.70,12.99,87.01,59.02,40.98,7.69,92.31,30.77,69.23


In [25]:
print("\nTEST TABLE ")
test_table = make_hate_nonhate_table(test_df)
test_table



TEST TABLE 


,% Hate → Muslim,% Non-Hate → Muslim,% Hate → Christian,% Non-Hate → Christian,% Hate → Jewish,% Non-Hate → Jewish,% Hate → Atheist,% Non-Hate → Atheist,% Hate → Buddhist,% Non-Hate → Buddhist
Muslim annotators,11.11,88.89,8.33,91.67,78.95,21.05,33.33,66.67,20.00,80.00
Christian annotators,15.15,84.85,9.38,90.62,70.91,29.09,14.94,85.06,15.52,84.48
Jewish annotators,11.63,88.37,17.24,82.76,72.92,27.08,50.00,50.00,33.33,66.67
Atheist annotators,16.67,83.33,8.96,91.04,75.17,24.83,14.71,85.29,24.14,75.86
Buddhist annotators,11.43,88.57,18.18,81.82,74.19,25.81,0.00,100.00,0.00,100.00


In [26]:
print("\nVALIDATION TABLE")
val_table = make_hate_nonhate_table(val_df)
val_table



VALIDATION TABLE


,% Hate → Muslim,% Non-Hate → Muslim,% Hate → Christian,% Non-Hate → Christian,% Hate → Jewish,% Non-Hate → Jewish,% Hate → Atheist,% Non-Hate → Atheist,% Hate → Buddhist,% Non-Hate → Buddhist
Muslim annotators,15.38,84.62,0.00,100.00,0.00,100.00,0.00,100.00,NaN,NaN
Christian annotators,41.23,58.77,6.32,93.68,51.07,48.93,8.51,91.49,12.90,87.10
Jewish annotators,47.50,52.50,5.88,94.12,88.89,11.11,0.00,100.00,100.00,0.00
Atheist annotators,43.23,56.77,7.30,92.70,59.57,40.43,26.92,73.08,36.36,63.64
Buddhist annotators,46.43,53.57,6.90,93.10,37.50,62.50,0.00,100.00,0.00,100.00


# Correct Splitting

In [27]:
annotator_religions = ["muslim", "christian", "jewish", "atheist", "buddhist"]
target_religions    = ["muslim", "christian", "jewish", "atheist", "buddhist"]

annotator_cols = [f"annotator_religion_{r}" for r in annotator_religions]
target_cols    = [f"target_religion_{r}" for r in target_religions]

df["hate_speech_score"] = pd.to_numeric(df["hate_speech_score"], errors="coerce")
df["is_hate_speech"] = (df["hate_speech_score"] > 0.5).astype(int)

df5 = df.copy()

df5["annotator_group"] = np.select(
    [df5[c].astype(int) == 1 for c in annotator_cols],
    annotator_religions,
    default=None
)

df5["target_group"] = np.select(
    [df5[c].astype(int) == 1 for c in target_cols],
    target_religions,
    default=None
)

df5 = df5.dropna(subset=["annotator_group", "target_group"])

print("Filtered rows:", len(df5))
print("Unique comments:", df5["comment_id"].nunique())


Filtered rows: 15856
Unique comments: 5428


In [28]:
print("Rows after filtering to 5×5 religions:", len(df5))
print("Annotator groups:", df5["annotator_group"].value_counts())
print("Target groups:", df5["target_group"].value_counts())


Rows after filtering to 5×5 religions: 15856
Annotator groups: annotator_group
christian    10050
atheist       4864
jewish         407
buddhist       348
muslim         187
Name: count, dtype: int64
Target groups: target_group
muslim       8309
christian    3648
jewish       3584
atheist       278
buddhist       37
Name: count, dtype: int64


In [29]:
df5["strat_bin"] = (
    df5["annotator_group"] + "|" +
    df5["target_group"] + "|" +
    df5["is_hate_speech"].astype(str)
)


In [31]:

comment_bin_counts = (
    df5.groupby(["comment_id", "strat_bin"])
        .size()
        .reset_index(name="count")
)


In [32]:

comment_bin_counts = comment_bin_counts.sort_values(
    ["comment_id", "count"],
    ascending=[True, False]
)

comment_bins = comment_bin_counts.drop_duplicates("comment_id")

comment_bins = comment_bins[["comment_id", "strat_bin"]]


In [33]:
bin_counts = comment_bins["strat_bin"].value_counts()

MIN_BIN_SIZE = 2

comment_bins["strat_bin_fixed"] = comment_bins["strat_bin"].where(
    comment_bins["strat_bin"].map(bin_counts) >= MIN_BIN_SIZE,
    "OTHER_BIN"
)

print("Number of original bins:", comment_bins["strat_bin"].nunique())
print("Number of fixed bins:", comment_bins["strat_bin_fixed"].nunique())
print("Rare bins collapsed into OTHER_BIN:",
      (comment_bins["strat_bin_fixed"] == "OTHER_BIN").sum())


Number of original bins: 44
Number of fixed bins: 38
Rare bins collapsed into OTHER_BIN: 7


In [34]:
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

train_ids = []
test_ids = []
val_ids = []

for bin_name, group in comment_bins.groupby("strat_bin_fixed"):
    ids = group["comment_id"].to_numpy().copy()
    rng.shuffle(ids)

    n = len(ids)
    n_train = int(round(n * 0.70))
    n_test  = int(round(n * 0.20))
    n_val   = n - n_train - n_test  # remainder goes to val (≈10%)

    train_ids.extend(ids[:n_train])
    test_ids.extend(ids[n_train:n_train+n_test])
    val_ids.extend(ids[n_train+n_test:])

train_ids = np.array(train_ids)
test_ids  = np.array(test_ids)
val_ids   = np.array(val_ids)

print("Train comments:", len(train_ids))
print("Test comments:", len(test_ids))
print("Val comments:", len(val_ids))

train_df = df5[df5["comment_id"].isin(train_ids)]
test_df  = df5[df5["comment_id"].isin(test_ids)]
val_df   = df5[df5["comment_id"].isin(val_ids)]

print("Train rows:", len(train_df))
print("Test rows:", len(test_df))
print("Val rows:", len(val_df))
print("Total rows after split:", len(train_df)+len(test_df)+len(val_df))
print("Original rows:", len(df5))


Train comments: 3795
Test comments: 1084
Val comments: 549
Train rows: 9581
Test rows: 3782
Val rows: 2493
Total rows after split: 15856
Original rows: 15856


In [35]:
print("Train∩Test:", len(set(train_ids) & set(test_ids)))
print("Train∩Val:", len(set(train_ids) & set(val_ids)))
print("Test∩Val:", len(set(test_ids) & set(val_ids)))



Train∩Test: 0
Train∩Val: 0
Test∩Val: 0


In [36]:
def make_table(data):
    table = pd.DataFrame()

    for a in annotator_religions:
        row = {}
        sub_a = data[data["annotator_group"] == a]

        for t in target_religions:
            sub = sub_a[sub_a["target_group"] == t]
            total = len(sub)

            if total > 0:
                hate = round(sub["is_hate_speech"].mean() * 100, 2)
                non = round(100 - hate, 2)
            else:
                hate = np.nan
                non = np.nan

            row[f"% Hate - {t.capitalize()}"] = hate
            row[f"% Non-Hate - {t.capitalize()}"] = non

        table = pd.concat([table, pd.DataFrame(row, index=[a.capitalize()+" annotators"])])

    return table


In [37]:
def make_table_b(df, title):
    s = df.style.format("{:.2f}").set_caption(title)
    hate_cols = [c for c in df.columns if "Hate -" in c]
    non_cols  = [c for c in df.columns if "Non-Hate -" in c]

    s = s.set_properties(subset=hate_cols, **{"font-weight": "bold"})
    s = s.set_properties(subset=non_cols, **{"color": "gray"})
    s = s.background_gradient(axis=None)
    return s

display(make_table_b(full_table, "Full Dataset: Hate vs Non-Hate (%)"))
display(make_table_b(train_table, "Train (70%)"))
display(make_table_b(test_table, "Test (20%)"))
display(make_table_b(val_table, "Validation (10%)"))


,% Hate → Muslim,% Non-Hate → Muslim,% Hate → Christian,% Non-Hate → Christian,% Hate → Jewish,% Non-Hate → Jewish,% Hate → Atheist,% Non-Hate → Atheist,% Hate → Buddhist,% Non-Hate → Buddhist
Muslim annotators,33.02,66.98,12.73,87.27,61.54,38.46,26.67,73.33,25.00,75.00
Christian annotators,40.79,59.21,13.29,86.71,63.01,36.99,18.93,81.07,23.80,76.20
Jewish annotators,45.45,54.55,13.13,86.87,69.13,30.87,35.71,64.29,33.33,66.67
Atheist annotators,43.23,56.77,10.27,89.73,66.23,33.77,18.54,81.46,20.00,80.00
Buddhist annotators,39.15,60.85,12.95,87.05,62.00,38.00,5.88,94.12,25.00,75.00


,% Hate → Muslim,% Non-Hate → Muslim,% Hate → Christian,% Non-Hate → Christian,% Hate → Jewish,% Non-Hate → Jewish,% Hate → Atheist,% Non-Hate → Atheist,% Hate → Buddhist,% Non-Hate → Buddhist
Muslim annotators,41.33,58.67,15.79,84.21,54.84,45.16,27.27,72.73,28.57,71.43
Christian annotators,48.27,51.73,17.45,82.55,60.78,39.22,21.59,78.41,27.16,72.84
Jewish annotators,56.35,43.65,13.21,86.79,65.22,34.78,40.00,60.00,25.00,75.00
Atheist annotators,50.31,49.69,12.05,87.95,62.40,37.60,17.80,82.20,16.67,83.33
Buddhist annotators,44.30,55.70,12.99,87.01,59.02,40.98,7.69,92.31,30.77,69.23


,% Hate → Muslim,% Non-Hate → Muslim,% Hate → Christian,% Non-Hate → Christian,% Hate → Jewish,% Non-Hate → Jewish,% Hate → Atheist,% Non-Hate → Atheist,% Hate → Buddhist,% Non-Hate → Buddhist
Muslim annotators,11.11,88.89,8.33,91.67,78.95,21.05,33.33,66.67,20.00,80.00
Christian annotators,15.15,84.85,9.38,90.62,70.91,29.09,14.94,85.06,15.52,84.48
Jewish annotators,11.63,88.37,17.24,82.76,72.92,27.08,50.00,50.00,33.33,66.67
Atheist annotators,16.67,83.33,8.96,91.04,75.17,24.83,14.71,85.29,24.14,75.86
Buddhist annotators,11.43,88.57,18.18,81.82,74.19,25.81,0.00,100.00,0.00,100.00


,% Hate → Muslim,% Non-Hate → Muslim,% Hate → Christian,% Non-Hate → Christian,% Hate → Jewish,% Non-Hate → Jewish,% Hate → Atheist,% Non-Hate → Atheist,% Hate → Buddhist,% Non-Hate → Buddhist
Muslim annotators,15.38,84.62,0.00,100.00,0.00,100.00,0.00,100.00,nan,nan
Christian annotators,41.23,58.77,6.32,93.68,51.07,48.93,8.51,91.49,12.90,87.10
Jewish annotators,47.50,52.50,5.88,94.12,88.89,11.11,0.00,100.00,100.00,0.00
Atheist annotators,43.23,56.77,7.30,92.70,59.57,40.43,26.92,73.08,36.36,63.64
Buddhist annotators,46.43,53.57,6.90,93.10,37.50,62.50,0.00,100.00,0.00,100.00


In [38]:
def make_count_table(data):
    counts = (
        data.groupby(["annotator_group", "target_group"])
            .size()
            .unstack(fill_value=0)
            .reindex(index=annotator_religions, columns=target_religions, fill_value=0)
    )
    counts.index = [x.capitalize() + " annotators" for x in counts.index]
    counts.columns = [x.capitalize() for x in counts.columns]
    return counts

print("FULL COUNTS ")
display(make_count_table(df5))

print("TRAIN COUNTS ")
display(make_count_table(train_df))

print("TEST COUNTS ")
display(make_count_table(test_df))

print("VAL COUNTS ")
display(make_count_table(val_df))


FULL COUNTS 


,Muslim,Christian,Jewish,Atheist,Buddhist
Muslim annotators,106,39,37,5,0
Christian annotators,5241,2402,2200,183,24
Jewish annotators,202,78,120,6,1
Atheist annotators,2580,1031,1161,81,11
Buddhist annotators,180,98,66,3,1


TRAIN COUNTS 


,Muslim,Christian,Jewish,Atheist,Buddhist
Muslim annotators,74,31,8,3,0
Christian annotators,3377,1798,779,124,17
Jewish annotators,124,53,52,3,1
Atheist annotators,1653,773,442,55,7
Buddhist annotators,111,74,21,1,0


TEST COUNTS 


,Muslim,Christian,Jewish,Atheist,Buddhist
Muslim annotators,22,7,13,2,0
Christian annotators,1215,384,702,41,4
Jewish annotators,52,15,36,1,0
Atheist annotators,630,175,374,18,1
Buddhist annotators,46,15,28,1,0


VAL COUNTS 


,Muslim,Christian,Jewish,Atheist,Buddhist
Muslim annotators,10,1,16,0,0
Christian annotators,649,220,719,18,3
Jewish annotators,26,10,32,2,0
Atheist annotators,297,83,345,8,3
Buddhist annotators,23,9,17,1,1


In [39]:
# # def mask_small_cells(percent_table, count_table, min_rows=30):
#     masked = percent_table.copy()
#     masked[count_table < min_rows] = np.nan
#     return masked


In [40]:
# full_table_masked  = mask_small_cells(full_table, make_count_table(df5), MIN_CELL_ROWS)
# train_table_masked = mask_small_cells(train_table, make_count_table(train_df), MIN_CELL_ROWS)
# test_table_masked  = mask_small_cells(test_table, make_count_table(test_df), MIN_CELL_ROWS)
# val_table_masked   = mask_small_cells(val_table, make_count_table(val_df), MIN_CELL_ROWS)

# display(full_table_masked.style.format("{:.2f}").set_caption("Full (masked small cells)"))
# display(train_table_masked.style.format("{:.2f}").set_caption("Train (masked small cells)"))
# display(test_table_masked.style.format("{:.2f}").set_caption("Test (masked small cells)"))
# display(val_table_masked.style.format("{:.2f}").set_caption("Val (masked small cells)"))


In [41]:
total_rows = len(df5)

train_pct = len(train_df) / total_rows * 100
test_pct  = len(test_df) / total_rows * 100
val_pct   = len(val_df) / total_rows * 100

print("ROW DISTRIBUTION ")
print(f"Train: {len(train_df)} rows ({train_pct:.2f}%)")
print(f"Test : {len(test_df)} rows ({test_pct:.2f}%)")
print(f"Val  : {len(val_df)} rows ({val_pct:.2f}%)")
print(f"Total after split: {len(train_df)+len(test_df)+len(val_df)}")
print(f"Original rows: {total_rows}")


ROW DISTRIBUTION 
Train: 9581 rows (60.43%)
Test : 3782 rows (23.85%)
Val  : 2493 rows (15.72%)
Total after split: 15856
Original rows: 15856


### Count version of table - 

In [42]:
def make_count_hate_nonhate_table(data):
    table = pd.DataFrame()

    for a in annotator_religions:
        row = {}
        sub_a = data[data["annotator_group"] == a]

        for t in target_religions:
            sub = sub_a[sub_a["target_group"] == t]

            hate_n = int(sub["is_hate_speech"].sum())
            total = len(sub)
            non_n = total - hate_n

            row[f"Hate (n) → {t.capitalize()}"] = hate_n
            row[f"Non-Hate (n) → {t.capitalize()}"] = non_n

        table = pd.concat([table, pd.DataFrame(row, index=[a.capitalize()+" annotators"])])

    return table


In [43]:
full_count_table  = make_count_hate_nonhate_table(df5)
train_count_table = make_count_hate_nonhate_table(train_df)
test_count_table  = make_count_hate_nonhate_table(test_df)
val_count_table   = make_count_hate_nonhate_table(val_df)


In [44]:
from IPython.display import display

def style_count_table(df, title):
    return (
        df.style
        .format("{:,.0f}")
        .set_caption(title)
        .set_table_styles([
            {"selector": "caption", "props": [("font-size", "16px"), ("font-weight", "bold")]},
            {"selector": "th", "props": [("background-color", "#f2f2f2"), ("font-weight", "bold")]}
        ])
        .background_gradient(cmap="Blues", axis=None)
    )


In [45]:
display(style_count_table(full_count_table,  "Full Dataset: Hate / Non-Hate (Counts)"))
display(style_count_table(train_count_table, "Train (70%) - Counts"))
display(style_count_table(test_count_table,  "Test (20%) - Counts"))
display(style_count_table(val_count_table,   "Validation (10%) - Counts"))


,Hate (n) → Muslim,Non-Hate (n) → Muslim,Hate (n) → Christian,Non-Hate (n) → Christian,Hate (n) → Jewish,Non-Hate (n) → Jewish,Hate (n) → Atheist,Non-Hate (n) → Atheist,Hate (n) → Buddhist,Non-Hate (n) → Buddhist
Muslim annotators,35,71,5,34,28,9,1,4,0,0
Christian annotators,"2,138","3,103",332,"2,070","1,594",606,36,147,5,19
Jewish annotators,89,113,10,68,89,31,1,5,0,1
Atheist annotators,"1,115","1,465",113,918,853,308,16,65,1,10
Buddhist annotators,69,111,11,87,46,20,0,3,1,0


,Hate (n) → Muslim,Non-Hate (n) → Muslim,Hate (n) → Christian,Non-Hate (n) → Christian,Hate (n) → Jewish,Non-Hate (n) → Jewish,Hate (n) → Atheist,Non-Hate (n) → Atheist,Hate (n) → Buddhist,Non-Hate (n) → Buddhist
Muslim annotators,28,46,3,28,3,5,1,2,0,0
Christian annotators,"1,502","1,875",222,"1,576",372,407,23,101,4,13
Jewish annotators,58,66,4,49,31,21,1,2,0,1
Atheist annotators,766,887,74,699,229,213,12,43,1,6
Buddhist annotators,46,65,8,66,9,12,0,1,0,0


,Hate (n) → Muslim,Non-Hate (n) → Muslim,Hate (n) → Christian,Non-Hate (n) → Christian,Hate (n) → Jewish,Non-Hate (n) → Jewish,Hate (n) → Atheist,Non-Hate (n) → Atheist,Hate (n) → Buddhist,Non-Hate (n) → Buddhist
Muslim annotators,7,15,2,5,10,3,0,2,0,0
Christian annotators,568,647,80,304,575,127,11,30,1,3
Jewish annotators,30,22,3,12,28,8,0,1,0,0
Atheist annotators,313,317,28,147,314,60,3,15,0,1
Buddhist annotators,21,25,1,14,22,6,0,1,0,0


,Hate (n) → Muslim,Non-Hate (n) → Muslim,Hate (n) → Christian,Non-Hate (n) → Christian,Hate (n) → Jewish,Non-Hate (n) → Jewish,Hate (n) → Atheist,Non-Hate (n) → Atheist,Hate (n) → Buddhist,Non-Hate (n) → Buddhist
Muslim annotators,0,10,0,1,15,1,0,0,0,0
Christian annotators,68,581,30,190,647,72,2,16,0,3
Jewish annotators,1,25,3,7,30,2,0,2,0,0
Atheist annotators,36,261,11,72,310,35,1,7,0,3
Buddhist annotators,2,21,2,7,15,2,0,1,1,0


### COUNT AND PERCENTAGE TOGETHER VERSION OF TABLE

In [46]:
def make_combined_table(data):
    table = pd.DataFrame()

    for a in annotator_religions:
        row = {}
        sub_a = data[data["annotator_group"] == a]

        for t in target_religions:
            sub = sub_a[sub_a["target_group"] == t]
            total = len(sub)

            hate_n = int(sub["is_hate_speech"].sum())
            non_n = total - hate_n

            if total > 0:
                hate_pct = hate_n / total * 100
                non_pct = non_n / total * 100

                hate_text = f"{hate_pct:.2f}% ({hate_n}/{total})"
                non_text  = f"{non_pct:.2f}% ({non_n}/{total})"
            else:
                hate_text = "—"
                non_text  = "—"

            row[f"% Hate - {t.capitalize()}"] = hate_text
            row[f"% Non-Hate - {t.capitalize()}"] = non_text

        table = pd.concat([table, pd.DataFrame(row, index=[a.capitalize() + " annotators"])])

    return table


In [47]:
full_combined  = make_combined_table(df5)
train_combined = make_combined_table(train_df)
test_combined  = make_combined_table(test_df)
val_combined   = make_combined_table(val_df)


In [48]:
from IPython.display import display

def style_combined(df, title):
    return (
        df.style
        .set_caption(title)
        .set_table_styles([
            {"selector": "caption", "props": [("font-size", "16px"), ("font-weight", "bold")]},
            {"selector": "th", "props": [("background-color", "#f2f2f2"), ("font-weight", "bold")]},
            {"selector": "td", "props": [("font-family", "Arial"), ("font-size", "12px")]}
        ])
    )

display(style_combined(full_combined,  "Full Dataset: Hate vs Non-Hate (%, counts)"))
display(style_combined(train_combined, "Train (70%): Hate vs Non-Hate (%, counts)"))
display(style_combined(test_combined,  "Test (20%): Hate vs Non-Hate (%, counts)"))
display(style_combined(val_combined,   "Validation (10%): Hate vs Non-Hate (%, counts)"))


,% Hate - Muslim,% Non-Hate - Muslim,% Hate - Christian,% Non-Hate - Christian,% Hate - Jewish,% Non-Hate - Jewish,% Hate - Atheist,% Non-Hate - Atheist,% Hate - Buddhist,% Non-Hate - Buddhist
Muslim annotators,33.02% (35/106),66.98% (71/106),12.82% (5/39),87.18% (34/39),75.68% (28/37),24.32% (9/37),20.00% (1/5),80.00% (4/5),—,—
Christian annotators,40.79% (2138/5241),59.21% (3103/5241),13.82% (332/2402),86.18% (2070/2402),72.45% (1594/2200),27.55% (606/2200),19.67% (36/183),80.33% (147/183),20.83% (5/24),79.17% (19/24)
Jewish annotators,44.06% (89/202),55.94% (113/202),12.82% (10/78),87.18% (68/78),74.17% (89/120),25.83% (31/120),16.67% (1/6),83.33% (5/6),0.00% (0/1),100.00% (1/1)
Atheist annotators,43.22% (1115/2580),56.78% (1465/2580),10.96% (113/1031),89.04% (918/1031),73.47% (853/1161),26.53% (308/1161),19.75% (16/81),80.25% (65/81),9.09% (1/11),90.91% (10/11)
Buddhist annotators,38.33% (69/180),61.67% (111/180),11.22% (11/98),88.78% (87/98),69.70% (46/66),30.30% (20/66),0.00% (0/3),100.00% (3/3),100.00% (1/1),0.00% (0/1)


,% Hate - Muslim,% Non-Hate - Muslim,% Hate - Christian,% Non-Hate - Christian,% Hate - Jewish,% Non-Hate - Jewish,% Hate - Atheist,% Non-Hate - Atheist,% Hate - Buddhist,% Non-Hate - Buddhist
Muslim annotators,37.84% (28/74),62.16% (46/74),9.68% (3/31),90.32% (28/31),37.50% (3/8),62.50% (5/8),33.33% (1/3),66.67% (2/3),—,—
Christian annotators,44.48% (1502/3377),55.52% (1875/3377),12.35% (222/1798),87.65% (1576/1798),47.75% (372/779),52.25% (407/779),18.55% (23/124),81.45% (101/124),23.53% (4/17),76.47% (13/17)
Jewish annotators,46.77% (58/124),53.23% (66/124),7.55% (4/53),92.45% (49/53),59.62% (31/52),40.38% (21/52),33.33% (1/3),66.67% (2/3),0.00% (0/1),100.00% (1/1)
Atheist annotators,46.34% (766/1653),53.66% (887/1653),9.57% (74/773),90.43% (699/773),51.81% (229/442),48.19% (213/442),21.82% (12/55),78.18% (43/55),14.29% (1/7),85.71% (6/7)
Buddhist annotators,41.44% (46/111),58.56% (65/111),10.81% (8/74),89.19% (66/74),42.86% (9/21),57.14% (12/21),0.00% (0/1),100.00% (1/1),—,—


,% Hate - Muslim,% Non-Hate - Muslim,% Hate - Christian,% Non-Hate - Christian,% Hate - Jewish,% Non-Hate - Jewish,% Hate - Atheist,% Non-Hate - Atheist,% Hate - Buddhist,% Non-Hate - Buddhist
Muslim annotators,31.82% (7/22),68.18% (15/22),28.57% (2/7),71.43% (5/7),76.92% (10/13),23.08% (3/13),0.00% (0/2),100.00% (2/2),—,—
Christian annotators,46.75% (568/1215),53.25% (647/1215),20.83% (80/384),79.17% (304/384),81.91% (575/702),18.09% (127/702),26.83% (11/41),73.17% (30/41),25.00% (1/4),75.00% (3/4)
Jewish annotators,57.69% (30/52),42.31% (22/52),20.00% (3/15),80.00% (12/15),77.78% (28/36),22.22% (8/36),0.00% (0/1),100.00% (1/1),—,—
Atheist annotators,49.68% (313/630),50.32% (317/630),16.00% (28/175),84.00% (147/175),83.96% (314/374),16.04% (60/374),16.67% (3/18),83.33% (15/18),0.00% (0/1),100.00% (1/1)
Buddhist annotators,45.65% (21/46),54.35% (25/46),6.67% (1/15),93.33% (14/15),78.57% (22/28),21.43% (6/28),0.00% (0/1),100.00% (1/1),—,—


,% Hate - Muslim,% Non-Hate - Muslim,% Hate - Christian,% Non-Hate - Christian,% Hate - Jewish,% Non-Hate - Jewish,% Hate - Atheist,% Non-Hate - Atheist,% Hate - Buddhist,% Non-Hate - Buddhist
Muslim annotators,0.00% (0/10),100.00% (10/10),0.00% (0/1),100.00% (1/1),93.75% (15/16),6.25% (1/16),—,—,—,—
Christian annotators,10.48% (68/649),89.52% (581/649),13.64% (30/220),86.36% (190/220),89.99% (647/719),10.01% (72/719),11.11% (2/18),88.89% (16/18),0.00% (0/3),100.00% (3/3)
Jewish annotators,3.85% (1/26),96.15% (25/26),30.00% (3/10),70.00% (7/10),93.75% (30/32),6.25% (2/32),0.00% (0/2),100.00% (2/2),—,—
Atheist annotators,12.12% (36/297),87.88% (261/297),13.25% (11/83),86.75% (72/83),89.86% (310/345),10.14% (35/345),12.50% (1/8),87.50% (7/8),0.00% (0/3),100.00% (3/3)
Buddhist annotators,8.70% (2/23),91.30% (21/23),22.22% (2/9),77.78% (7/9),88.24% (15/17),11.76% (2/17),0.00% (0/1),100.00% (1/1),100.00% (1/1),0.00% (0/1)
